<a href="https://colab.research.google.com/github/PratikChakrabortyaz/Hugging-Face-/blob/main/Audio%20Course/Audio_Course_Unit_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets[audio] evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [12]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.2 MB/s eta 0:00:00


In [17]:
!pip install -q huggingface_hub tensorboard

In [3]:
from datasets import load_dataset, DatasetDict

# Load only the available train split
minds = load_dataset("PolyAI/minds14", "en-US", split="train")

# Split first 450 for training, the rest for evaluation
train_dataset = minds.select(range(450))
test_dataset = minds.select(range(450, len(minds)))

# Wrap into DatasetDict (optional)
minds = DatasetDict({"train": train_dataset, "test": test_dataset})

print(minds)




DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 450
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 113
    })
})


In [4]:
from transformers import WhisperProcessor

# Load processor with task as ASR (Automatic Speech Recognition)
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="english", task="transcribe")



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [8]:
from datasets import Audio

# Ensure dataset audio is resampled to Whisper's required sampling rate (16kHz)
sampling_rate = processor.feature_extractor.sampling_rate
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))

def preprocess_function(example):
    audio = example["audio"]

    # Extract features from audio & tokenize target transcription
    inputs = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["transcription"]  # Change from "text" to "transcription"
    )

    # Store input length
    inputs["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return inputs

train_dataset = train_dataset.map(
    preprocess_function,
    remove_columns=["path", "audio"],  # Only remove columns that exist
    num_proc=1
)

test_dataset = test_dataset.map(
    preprocess_function,
    remove_columns=["path", "audio"],  # Only remove columns that exist
    num_proc=1
)




Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

In [9]:
max_input_length = 30.0

# Function to filter long audio samples
def is_audio_in_length_range(length):
    return length < max_input_length

train_dataset = train_dataset.filter(is_audio_in_length_range, input_columns=["input_length"])
test_dataset = test_dataset.filter(is_audio_in_length_range, input_columns=["input_length"])

print(f"Train dataset size after filtering: {len(train_dataset)}")
print(f"Test dataset size after filtering: {len(test_dataset)}")


Filter:   0%|          | 0/450 [00:00<?, ? examples/s]

Filter:   0%|          | 0/113 [00:00<?, ? examples/s]

Train dataset size after filtering: 445
Test dataset size after filtering: 113


In [10]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # Split inputs and labels
        input_features = [{"input_features": feature["input_features"][0]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Get tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Replace padding tokens with -100 (ignored in loss calculation)
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # Remove BOS token
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

# Initialize data collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [13]:
import evaluate

# Load the Word Error Rate metric
metric = evaluate.load("wer")

# Define function to compute WER
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace padding tokens
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # Convert token IDs back to text
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # Compute WER
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [14]:
from transformers import WhisperForConditionalGeneration

# Load pre-trained Whisper Tiny model
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

# Disable cache during training (incompatible with gradient checkpointing)
model.config.use_cache = False


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

In [15]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tiny-minds14",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # Adjust based on GPU memory
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=500,  # Increase if GPU allows
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
from huggingface_hub import notebook_login

notebook_login()


In [19]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)


<ipython-input-19-e9f5025f1559>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [20]:
trainer.train()


Step,Training Loss,Validation Loss,Wer
500,0.001700,0.589034,32.078964


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183

TrainOutput(global_step=500, training_loss=0.3460448551401496, metrics={'train_runtime': 607.4396, 'train_samples_per_second': 6.585, 'train_steps_per_second': 0.823, 'total_flos': 9.788468723712e+16, 'train_loss': 0.3460448551401496, 'epoch': 8.928571428571429})

In [23]:
kwargs = {
    "dataset_tags": "PolyAI/minds14",
    "dataset": "PolyAI MINDS-14",  # A 'pretty' name for the training dataset
    "language": "en",  # Corrected to lowercase ISO 639-1 code
    "model_name": "Whisper Tiny - Fine-tuned on MINDS-14",  # A 'pretty' name for your model
    "finetuned_from": "openai/whisper-tiny",
    "tasks": "automatic-speech-recognition",
}


In [24]:
trainer.push_to_hub(**kwargs)


CommitInfo(commit_url='https://huggingface.co/hp1318/whisper-tiny-minds14/commit/efb31611708ddab72e5b2b0896ccf7a4ee91ecb4', commit_message='End of training', commit_description='', oid='efb31611708ddab72e5b2b0896ccf7a4ee91ecb4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hp1318/whisper-tiny-minds14', endpoint='https://huggingface.co', repo_type='model', repo_id='hp1318/whisper-tiny-minds14'), pr_revision=None, pr_num=None)